Based on:

# Tacotron2: WaveNet-basd text-to-speech demo

- Tacotron2 (mel-spectrogram prediction part): https://github.com/Rayhane-mamah/Tacotron-2
- WaveNet: https://github.com/r9y9/wavenet_vocoder

This is a proof of concept for Tacotron2 text-to-speech synthesis. Models used here were trained on [LJSpeech dataset](https://keithito.com/LJ-Speech-Dataset/).

**Notice**: The waveform generation is super slow since it implements naive autoregressive generation. It doesn't use parallel generation method described in [Parallel WaveNet](https://arxiv.org/abs/1711.10433). 

**Estimated time to complete**: 2 ~ 3 hours.

To Do: migrate to
https://github.com/azraelkuan/parallel_wavenet_vocoder

In [0]:
# !pip install -U -q PyDrive
# import os
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# # 1. Authenticate and create the PyDrive client.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

# # choose a local (colab) directory to store the data.
# local_download_path = os.path.expanduser('/content/sounds')
# try:
#   os.makedirs(local_download_path)
# except: pass

# # obtain the training data from google drive
# download = drive.CreateFile({'id': '1lKb0ORrlnTkFG9PXRvU7LYW2svy6K9w_'})
# name = os.path.join(local_download_path, 'split-mp3.zip')
# download.GetContentFile(name)


In [0]:
# !unzip -q ./sounds/split-mp3.zip

In [0]:
# os.chdir("/content/")
# !mv checkpoint_step000004686.pth ./checkpoint/

## Setup

### Install dependencies

In [0]:
# import os
# from os.path import exists, join, expanduser
# os.chdir(expanduser("~"))
# wavenet_dir = "DTT_wavenet_pytorch"
# taco2_dir = "Tacotron-2"
# !rm -r $taco2_dir
# !rm -r $wavenet_dir

In [0]:
import os
from os.path import exists, join, expanduser

os.chdir(expanduser("~"))

wavenet_dir = "DTT_wavenet_pytorch"
if not exists(wavenet_dir):
  ! git clone https://github.com/popo0293/$wavenet_dir
    
taco2_dir = "Tacotron-2"
if not exists(taco2_dir):
  ! git clone https://github.com/r9y9/$taco2_dir    
  ! cd $taco2_dir && git checkout -B wavenet3 origin/wavenet3

Cloning into 'DTT_wavenet_pytorch'...
remote: Enumerating objects: 100, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 100 (delta 23), reused 92 (delta 15), pack-reused 0
Receiving objects: 100% (100/100), 249.57 KiB | 692.00 KiB/s, done.
Resolving deltas: 100% (23/23), done.
Cloning into 'Tacotron-2'...
remote: Enumerating objects: 570, done.
remote: Total 570 (delta 0), reused 0 (delta 0), pack-reused 570
Receiving objects: 100% (570/570), 8.05 MiB | 4.92 MiB/s, done.
Resolving deltas: 100% (355/355), done.
Branch 'wavenet3' set up to track remote branch 'wavenet3' from 'origin'.
Switched to a new branch 'wavenet3'


In [0]:
# !pip install numpy==1.16.2

In [0]:
# Install dependencies
! pip install -q --upgrade "tensorflow<=1.9.0"

os.chdir(join(expanduser("~"), taco2_dir))
! pip install -q -r requirements.txt

os.chdir(join(expanduser("~"), wavenet_dir))
! pip install -q -e '.[train]'
# ! pip install -q -r requires.txt

    100% |████████████████████████████████| 51.1MB 935kB/s 
    100% |████████████████████████████████| 3.3MB 10.3MB/s 
    100% |████████████████████████████████| 573kB 20.1MB/s 
magenta 0.3.19 has requirement tensorflow>=1.12.0, but you'll have tensorflow 1.9.0 which is incompatible.
datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
    100% |████████████████████████████████| 133kB 4.2MB/s 
    100% |████████████████████████████████| 61kB 21.4MB/s 
    100% |████████████████████████████████| 1.5MB 1.7MB/s 
    100% |████████████████████████████████| 14.6MB 2.4MB/s 
    100% |████████████████████████████████| 235kB 19.9MB/s 
magenta 0.3.19 has requirement librosa>=0.6.2, but you'll have librosa 0.5.1 which is incompatible.
magenta 0.3.19 has requirement tensorflow>=1.12.0, but you'll have tensorflow 1.9.0 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompati

In [0]:
# !pip install -q cadl

In [0]:
# import os
# import sys
# import subprocess
# from glob import glob
# import numpy as np
# import tensorflow as tf
# from cadl import wavenet, vctk
# from cadl import wavenet_utils as wnu
# from cadl.utils import sample_categorical
# from scipy.io import wavfile

In [0]:
# def get_dataset(saveto='/content/split-mp3', convert_mp3_to_16khzwav=False):
#     """Convert MP3 files in 'saveto' directory to wav files.
#     subfolders under the 'saveto' directory are considered chapters
#     Each file name should be formatted CHAPTERNAME-UTTERANCE-DESCRIPTION.mp3
#     ffmpeg must be installed to convert the files.
#     Parameters
#     ----------
#     saveto : str, optional
#         Directory to save the resulting dataset ['sounds']
#     convert_to_16khz : bool, optional
#         Description
#     Returns
#     -------
#         dataset
#     """
#     if not os.path.exists(saveto):
#         sys.exit("Error: '" + saveto + "' folder does not exist")

#     wavs = glob('{}/**/*.wav'.format(saveto), recursive=True)
#     if not wavs and convert_mp3_to_16khzwav:
#         wavs = glob('{}/**/*.mp3'.format(saveto), recursive=True)
#         for wav_i in wavs:
#             subprocess.check_call(
#                 ['ffmpeg', '-i', wav_i, '-f', 'wav', '-ac', '1', '-ar', '16000', '-y', '%s.16khz.wav' % wav_i])

#     wavs = glob('{}/**/*.wav'.format(saveto), recursive=True)

#     if not wavs:
#         sys.exit("Error: No 16khz wav files were found in '" + saveto + "'")        

#     dataset = []
#     for wav_i in wavs:
#         chapter_i, utter_i = wav_i.split('/')[-2:]
#         dataset.append({
#             'name': wav_i,
#             'chapter': chapter_i,
#             'utterance': utter_i.split('-')[-2].strip('.wav')})
#     return dataset

In [0]:
# get_dataset(convert_mp3_to_16khzwav=True)

In [0]:
# !find /content/split-mp3/ -name '*.mp3' -delete

In [0]:
# os.chdir(join('/content/split-mp3/'))
# !ls -1 > ../allfiles.txt

In [0]:
os.chdir("/content/")
if not exists("preprocessed"):
  ! curl -O -L "https://www.dropbox.com/s/u2y4s00c40lhrpk/preprocessed.zip?dl=0"
  ! mv preprocessed.zip?dl=0 preprocessed.zip
  ! unzip -q preprocessed.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   990    0   990    0     0    638      0 --:--:--  0:00:01 --:--:--     0
100  155M  100  155M    0     0  24.1M      0  0:00:06  0:00:06 --:--:-- 39.7M


In [0]:
os.chdir("/content/")
if not exists("preprocessed"):
  !mv ./content/preprocessed ./preprocessed
  !rmdir ./content

In [0]:
# !python preprocess.py dtt /content /content/preprocessed --preset ./presets/dtt_speech.json

In [0]:
# !mv /dtt_speech.json ./presets/

In [0]:
# os.chdir("/content/")
# !rm -r checkpoint


In [0]:
os.chdir("/content/")
if not exists("checkpoint"):
  ! mkdir checkpoint
  os.chdir("/content/checkpoint/")
  ! curl -O -L "https://www.dropbox.com/s/yn6hsyye2q1d71v/checkpoint_step000018900.pth?dl=0"
  ! mv checkpoint_step000018900.pth?dl=0 checkpoint_step000018900.pth

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   990    0   990    0     0    540      0 --:--:--  0:00:01 --:--:--     0
100  282M  100  282M    0     0  24.9M      0  0:00:11  0:00:11 --:--:-- 29.9M


In [0]:
# os.chdir("/content/")
# !mkdir checkpoint
# os.chdir("/content/checkpoint/")
# !mv ../checkpoint_step000002872.pth . 

In [0]:
os.chdir(join(expanduser("~"), wavenet_dir))
!python train.py --data-root /content/preprocessed --checkpoint /content/checkpoint/checkpoint_step000018900.pth --hparams='batch_size=3,' --checkpoint-dir /content/checkpoint --preset ./presets/dtt_speech.json

Using TensorFlow backend.
Command line args:
 {'--checkpoint': '/content/checkpoint/checkpoint_step000018900.pth',
 '--checkpoint-dir': '/content/checkpoint',
 '--data-root': '/content/preprocessed',
 '--help': False,
 '--hparams': 'batch_size=3,',
 '--log-event-path': None,
 '--preset': './presets/dtt_speech.json',
 '--reset-optimizer': False,
 '--restore-parts': None,
 '--speaker-id': None}
Hyperparameters:
  adam_beta1: 0.9
  adam_beta2: 0.999
  adam_eps: 1e-08
  allow_clipping_in_normalization: True
  amsgrad: False
  batch_size: 3
  builder: wavenet
  checkpoint_interval: 10000
  cin_channels: 80
  clip_thresh: -1
  dropout: 0.050000000000000044
  ema_decay: 0.9999
  exponential_moving_average: True
  fft_size: 1024
  fmax: 7600
  fmin: 125
  frame_shift_ms: None
  freq_axis_kernel_size: 3
  gate_channels: 512
  gin_channels: -1
  hop_size: 256
  initial_learning_rate: 0.001
  input_type: raw
  kernel_size: 3
  layers: 24
  legacy: True
  log_scale_min: -32.23619130191664
  lr_sch

In [0]:
# !zip /content/DTTpytorch.zip -r /content/checkpoint 

In [0]:
# !zip /content/preprocessed.zip -r /content/preprocessed

In [0]:
import torch
import tensorflow
tensorflow.__version__

'1.9.0'

### Download pretrained models

#### Tacotron2 (mel-spectrogram prediction part)

In [0]:
os.chdir(join(expanduser("~"), taco2_dir))
! mkdir -p logs-Tacotron
if not exists("logs-Tacotron/pretrained"):
  ! curl -O -L "https://www.dropbox.com/s/vx7y4qqs732sqgg/pretrained.tar.gz"
  ! tar xzvf pretrained.tar.gz
  ! mv pretrained logs-Tacotron

#### WaveNet

In [0]:
# os.chdir('/content/checkpoint')
wn_preset = "/root/DTT_wavenet_pytorch/presets/dtt_speech.json"
wn_checkpoint_path = "/content/checkpoint/checkpoint_step000010000.pth"

# if not exists(wn_preset):
#   !curl -O -L "https://www.dropbox.com/s/0vsd7973w20eskz/20180510_mixture_lj_checkpoint_step000320000_ema.json"
# if not exists(wn_checkpoint_path):
#   !curl -O -L "https://www.dropbox.com/s/zdbfprugbagfp2w/20180510_mixture_lj_checkpoint_step000320000_ema.pth"

## Input texts to be synthesized

Choose your favorite sentences :)

In [0]:
os.chdir(join(expanduser("~"), taco2_dir))

In [0]:
%%bash
cat << EOS > text_list.txt
Hope you like it.
EOS

cat text_list.txt

Hope you like it.


## Mel-spectrogram prediction by Tacoron2

In [0]:
os.chdir(join(expanduser("~"), taco2_dir))
# Remove old files if exist
! rm -rf tacotron_output
! python synthesize.py --model='Tacotron' --mode='eval' \
  --hparams='symmetric_mels=False,max_abs_value=4.0,power=1.1,outputs_per_step=1' \
  --text_list=./text_list.txt

ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'
ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'
ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'
ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'
loaded model at logs-Tacotron/pretrained/model.ckpt-189500
Hyperparameters:
  allow_clipping_in_normalization: True
  attention_dim: 128
  attention_filters: 32
  attention_kernel: (31,)
  cleaners: english_cleaners
  cumulative_weights: True
  decoder_layers: 2
  decoder_lstm_units: 1024
  embedding_dim: 512
  enc_conv_channels: 512
  enc_conv_kernel_size: (5,)
  enc_conv_num_layers: 3
  encoder_lstm_units: 256
  fft_size: 1024
  fmax: 7600
  fmin: 125
  frame_shift_ms: None
  griffin_lim_iters: 60
  hop_size: 256
  impute_finished: False
  input_type: raw
  log_scale_min: -32.23619130191664
  mask_encoder: False
  mask_finished: False
  max_abs_value: 4.0
  max_iters: 2500
  min_level_db: -100
  num_freq: 513
  num_m

## Waveform synthesis by WaveNet

In [0]:
import librosa.display
import IPython
from IPython.display import Audio
import numpy as np
import torch

In [0]:
os.chdir(join(expanduser("~"), wavenet_dir))

# Setup WaveNet vocoder hparams
from hparams import hparams
with open(wn_preset) as f:
    hparams.parse_json(f.read())

# Setup WaveNet vocoder
from train import build_model
from synthesis import wavegen
import torch

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

model = build_model().to(device)

print("Load checkpoint from {}".format(wn_checkpoint_path))
checkpoint = torch.load(wn_checkpoint_path)
model.load_state_dict(checkpoint["state_dict"])

Load checkpoint from /content/checkpoint/checkpoint_step000010000.pth


In [0]:
from glob import glob
from tqdm import tqdm

with open("../Tacotron-2/tacotron_output/eval/map.txt") as f:
  maps = f.readlines()
maps = list(map(lambda x:x[:-1].split("|"), maps))
# filter out invalid ones
maps = list(filter(lambda x:len(x) == 2, maps))

print("List of texts to be synthesized")
for idx, (text,_) in enumerate(maps):
  print(idx, text)

List of texts to be synthesized
0 Hope you like it.


### Waveform generation

**Note**: This will takes hours to finish depending on the number and lenght of texts. Try short sentences first if you would like to see samples quickly.

In [0]:
waveforms = []

for idx, (text, mel) in enumerate(maps):
  print("\n", idx, text)
  mel_path = join("../Tacotron-2", mel)
  c = np.load(mel_path)
  if c.shape[1] != hparams.num_mels:
    np.swapaxes(c, 0, 1)
  # Range [0, 4] was used for training Tacotron2 but WaveNet vocoder assumes [0, 1]
  c = np.interp(c, (0, 4), (0, 1))
 
  # Generate
  waveform = wavegen(model, c=c, fast=True, tqdm=tqdm)
  
  waveforms.append(waveform)

  # Audio
  IPython.display.display(Audio(waveform, rate=hparams.sample_rate))

  0%|          | 7/19456 [00:00<05:12, 62.33it/s]


 0 Hope you like it.


100%|██████████| 19456/19456 [05:30<00:00, 58.87it/s]


## Summary: audio samples

In [0]:
for idx, (text, mel) in enumerate(maps):
  print(idx, text)
  IPython.display.display(Audio(waveforms[idx], rate=hparams.sample_rate))

For more information, please visit https://github.com/r9y9/wavenet_vocoder. More samples can  be  found at https://r9y9.github.io/wavenet_vocoder/. 